In [1]:
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.autograd import Variable
import os

In [ ]:
def flatten(tensor):
    """Flattens a given tensor such that the channel axis is first.
    The shapes are transformed as follows:
       (N, C, D, H, W) -> (C, N * D * H * W)
    """
    C = tensor.size(1)
    # new axis order
    axis_order = (1, 0) + tuple(range(2, tensor.dim()))
    # Transpose: (N, C, D, H, W) -> (C, N, D, H, W)
    transposed = tensor.permute(axis_order)
    # Flatten: (C, N, D, H, W) -> (C, N * D * H * W)
    return transposed.contiguous().view(C, -1)

In [ ]:
class DiceLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.epsilon = 1e-5

    def forward(self, output, target):
        assert output.size() == target.size(), "'input' and 'target' must have the same shape"
        output = F.softmax(output, dim=1)
        output = flatten(output)
        target = flatten(target)
        # intersect = (output * target).sum(-1).sum() + self.epsilon
        # denominator = ((output + target).sum(-1)).sum() + self.epsilon

        intersect = (output * target).sum(-1)
        denominator = (output + target).sum(-1)
        dice = intersect / denominator
        dice = torch.mean(dice)
        return 1 - dice
        # return 1 - 2. * intersect / denominator

In [ ]:
class CrossEntropyLoss(nn.Module):
  def __init__(self, background_label_value = 255):
    super().__init__()
    self.background = background_label_value
  
  def forward(self, prediction, target, weight = None):
    
    batch_size, number_classes, height, width = prediction.size()
    target_mask = (target >= 0) * (target != self.background)
    target = target[target_mask]

    if not target.data.dim():
      return Variable(torch.zeros(1))

    prediction = prediction.transpose(1, 2).transpose(2, 3).contiguous()
    prediction = prediction[target_mask.view(batch_size, height, width, 1).repeat(1, 1, 1, number_classes)].view(-1, number_classes)

    loss = F.cross_entropy(prediction, target, weight=weight)
    return loss
    


In [ ]:
def find_frequency_class(data_loader):
  density_categories = {}
  for _, label in data_loader:
    array_pixels = label.flatten().numpy()
    categories_contained = np.unique(array_pixels)

    for category in categories_contained:
      if str(catergory) in density_categories.keys():
        density_category[str(category)] += 1
      else:
        density_category[str(category)] = 1
  return density_categories




In [ ]:
class FocalLoss(nn.Module):
  """Implementation of Facal Loss"""
  def __init__(self, weight=None, gamma=2, reduction="mean", ignore_index=255):
      super(FocalLoss, self).__init__()
      self.weighted_cs = nn.CrossEntropyLoss(weight=weight, reduction="none", ignore_index=ignore_index)
      self.cs = nn.CrossEntropyLoss(reduction="none", ignore_index=ignore_index)
      self.gamma = gamma
      self.reduction = reduction
      
  def forward(self, predicted, target):
      """
      predicted: [batch_size, n_classes]
      target: [batch_size]
      """
      pt = 1/torch.exp(self.cs(predicted,target))
      #shape: [batch_size]
      entropy_loss = self.weighted_cs(predicted, target)
      #shape: [batch_size]
      focal_loss = ((1-pt)**self.gamma)*entropy_loss
      #shape: [batch_size]
      if self.reduction =="none":
          return focal_loss
      elif self.reduction == "mean":
          return focal_loss.mean()
      else:
          return focal_loss.sum()

In [ ]:
############## COMPUTE WEIGHTS #####################
# Compute weights for each classes of the dataset  #
# Receive in input:                                #             
# savedir: the directory where to save the weights #
# dataloader: the train dataloader                 #
# num_classes: the number of total classes         #                      
####################################################
def calculate_weigths_labels(savedir, dataloader, num_classes):
    # Create an instance from the data loader
    z = np.zeros((num_classes,))
    # Initialize tqdm
    print('Calculating classes weights')
    for step, (images, labels) in enumerate(dataloader):
        x = images
        y = labels.detach().cpu().numpy()
        mask = (y >= 0) & (y < num_classes)
        labels = y[mask].astype(np.uint8)
        count_l = np.bincount(labels, minlength=num_classes)
        z += count_l


    total_frequency = np.sum(z)
    class_weights = []
    for frequency in z:
        class_weight = 1 / (np.log(1.02 + (frequency / total_frequency)))
        class_weights.append(float(class_weight))
    ret = np.array(class_weights)
    classes_weights_path = os.path.join(savedir, 'classes_weights.npy')
    np.save(classes_weights_path, ret)
    
    ret = torch.from_numpy(ret).type(torch.FloatTensor)
    return ret